### Setting up a Lambda function that creates a new IAM user with administrator access

In [ ]:
import boto3

def get_account_id():
    sts_client = boto3.client('sts')
    response = sts_client.get_caller_identity()
    
    return response['Account']

In [ ]:
function_name = 'superadmin-iam-creator'
role_name = '<LAMBDA ROLE NAME>'
role_arn = f'arn:aws:iam::{get_account_id()}:role/{role_name}'

In [ ]:
import boto3
import io
import zipfile


def create_function(function_name, code, role_arn, handler='lambda_function.handler', runtime='python3.8'):
    lambda_client = boto3.client('lambda')
    
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w') as zf:
        zf.writestr('lambda_function.py', code)

    zip_buffer.seek(0)
    
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime=runtime,
        Role=role_arn,
        Handler=handler,
        Code={
            'ZipFile': zip_buffer.read()
        }
    )

    return response['FunctionArn']

In [ ]:
lambda_code = '''
import boto3
import secrets
import string
from botocore.exceptions import ClientError

iam_client = boto3.client('iam')

def generate_random_password(length=16):
    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(secrets.choice(characters) for _ in range(length))
    return password


def handler(event, context):
    new_username = 'new-iam-user'
    
    try:
        response = iam_client.create_user(UserName=new_username)
        print("IAM user created successfully.")
    except ClientError as e:
        print("Error creating IAM user:", e)
        return {
            'statusCode': 500,
            'body': 'Error creating IAM user'
        }
    
    try:
        iam_client.attach_user_policy(
            UserName=new_username,
            PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
        )
        print("Admin access policy attached to the IAM user.")
    except ClientError as e:
        print("Error attaching admin access policy:", e)
        return {
            'statusCode': 500,
            'body': 'Error attaching admin access policy'
        }
    
    try:
        response = iam_client.create_access_key(UserName=new_username)
        access_key = response['AccessKey']['AccessKeyId']
        secret_key = response['AccessKey']['SecretAccessKey']
        print("Access key pair generated for the IAM user.")
    except ClientError as e:
        print("Error generating access key pair:", e)
        return {
            'statusCode': 500,
            'body': 'Error generating access key pair'
        }
    
    password = generate_random_password()
    
    try:
        response = iam_client.create_login_profile(
            UserName=new_username,
            Password=password,
            PasswordResetRequired=False
        )
        print("Login profile created for the IAM user.")
    except ClientError as e:
        print("Error creating login profile:", e)
        return {
            'statusCode': 500,
            'body': 'Error creating login profile'
        }
    
    return {
        'statusCode': 200,
        'body': {
            'username': new_username,
            'access_key': access_key,
            'secret_key': secret_key,
            'password': password
        }
    }
'''



create_function(function_name=function_name,
                code=lambda_code,
                role_arn=role_arn)

### Invoking the created Lambda function

In [ ]:
import boto3
import json


def invoke_function(function_name, payload = {}):
    lambda_client = boto3.client('lambda')

    response = lambda_client.invoke(
        FunctionName=function_name,
        Payload=bytes(json.dumps(payload), 'utf-8')
    )

    result = response['Payload'].read().decode('utf-8')
    return result

In [ ]:
result = invoke_function(function_name)
result

### Deleting the Lambda function

In [ ]:
import boto3
from botocore.exceptions import ClientError


def delete_function(function_name):
    lambda_client = boto3.client('lambda')
    success = False

    try:
        lambda_client.delete_function(FunctionName=function_name)
        success = True
    except ClientError as e:
        print(f"Error deleting Lambda function '{function_name}': {e}")
        
    return success

In [ ]:
delete_function(function_name)